In [ ]:
import tqdm
import matplotlib.pyplot as plt
import numpy as np
import rasterio
import firedrake
from firedrake import Constant, sqrt, inner, grad, dx, ds
import icepack
from icepack.constants import ice_density as ρ_I, gravity as g

Load in the work from the previous notebook.

In [ ]:
filename = "modern_state.h5"
with firedrake.CheckpointFile(filename, "r") as chk:
    mesh = chk.load_mesh()
    C = chk.load_function(mesh, "friction")
    u_0 = chk.load_function(mesh, "velocity")

In [ ]:
coords = mesh.coordinates.dat.data_ro[:]
delta = 10e3
xmin, xmax = coords[:, 0].min() - delta, coords[:, 0].max() + delta
ymin, ymax = coords[:, 1].min() - delta, coords[:, 1].max() + delta

In [ ]:
image_filename = icepack.datasets.fetch_mosaic_of_antarctica()
with rasterio.open(image_filename, "r") as image_file:
    height, width = image_file.height, image_file.width
    transform = image_file.transform
    window = rasterio.windows.from_bounds(
        left=xmin,
        bottom=ymin,
        right=xmax,
        top=ymax,
        transform=transform,
    )
    image = image_file.read(indexes=1, window=window, masked=True)

In [ ]:
def subplots(*args, **kwargs):
    fig, axes = plt.subplots()
    axes.set_aspect("equal")
    xmin, ymin, xmax, ymax = rasterio.windows.bounds(window, transform)
    axes.imshow(
        image,
        cmap="Greys_r",
        vmin=12e3,
        vmax=16.38e3,
        extent=(xmin, xmax, ymin, ymax),
    )

    return fig, axes

In [ ]:
Q = firedrake.FunctionSpace(mesh, family="CG", degree=2)
V = firedrake.VectorFunctionSpace(mesh, family="CG", degree=2)

In [ ]:
bedmachine_filename = icepack.datasets.fetch_bedmachine_antarctica()
thickness_filename = f"netcdf:{bedmachine_filename}:thickness"
with rasterio.open(thickness_filename, "r") as thickness_file:
    h_0 = icepack.interpolate(thickness_file, Q)

surface_filename = f"netcdf:{bedmachine_filename}:surface"
with rasterio.open(surface_filename, "r") as surface_file:
    s_0 = icepack.interpolate(surface_file, Q)

In [ ]:
b = firedrake.interpolate(s_0 - h_0, Q)

This is the accumulation rate that we calculated in the last notebook that we'd need in order to roughly keep the system in steady state.

In [ ]:
a = Constant(0.1)

We'll use the same values for the temperature as before.

In [ ]:
T = Constant(260.0)
A = icepack.rate_factor(T)

But we'll do something a little tricky with the terminus stress.
When terminus BCs are applied, the model assumes that this boundary is adjacent to ocean water.
In our case, there's more ice downstream, and we want to apply backpressure consistent with the thickness of this downstream ice.

In [ ]:
def terminus(**kwargs):
    u = kwargs["velocity"]
    h = kwargs["thickness"]
    s = kwargs["surface"]
    h_D = kwargs["thickness_downstream"]
    
    τ_I = 0.5 * ρ_I * g * h**2
    τ_D = 0.5 * ρ_I * g * h_D**2

    ν = firedrake.FacetNormal(mesh)
    return (τ_I - τ_D) * inner(u, ν)

In [ ]:
model = icepack.models.IceStream(terminus=terminus)
opts = {
    "dirichlet_ids": [1, 2, 3, 5, 6, 7],
    "diagnostic_solver_type": "petsc",
    "diagnostic_solver_parameters": {
        "snes_max_it": 100,
        "snes_type": "newtontr",
        "ksp_type": "gmres",
        "pc_type": "lu",
        "pc_factor_mat_solver_type": "mumps",
    },
}
solver = icepack.solvers.FlowSolver(model, **opts)

In [ ]:
s = s_0.copy(deepcopy=True)
h = h_0.copy(deepcopy=True)
h_D = h_0.copy(deepcopy=True)

Having modified the terminus pressure calculation, we just pass whatever value we want for the downstream thickness.
We're keeping it constant for now but you can make it change in time if you want.

In [ ]:
u = solver.diagnostic_solve(
    velocity=u_0,
    thickness=h,
    surface=s,
    fluidity=A,
    friction=C,
    thickness_downstream=h_D,
)

Now let's actually try to simulate things for a while.
I started trying this with a timestep of 1 year.
Things can explode if you set that value too high, so I worked it down to a timestep of 2 months by trial and error.
After the system propagates out some initial transients, we'll dial this back up to a larger value for efficiency's sake.

In [ ]:
hs = [h.copy(deepcopy=True)]
ss = [s.copy(deepcopy=True)]
us = [u.copy(deepcopy=True)]

In [ ]:
final_time = 40.0
dt = 1 / 6
num_steps = int(final_time / dt)

for step in tqdm.trange(num_steps):
    h = solver.prognostic_solve(
        dt,
        thickness=h,
        velocity=u,
        accumulation=a,
        thickness_inflow=h_0,
    )
    h.interpolate(firedrake.max_value(1.0, h))

    s = icepack.compute_surface(thickness=h, bed=b)

    u = solver.diagnostic_solve(
        velocity=u,
        thickness=h,
        surface=s,
        fluidity=A,
        friction=C,
        thickness_downstream=h_D,
    )
    
    hs.append(h.copy(deepcopy=True))
    ss.append(s.copy(deepcopy=True))
    us.append(u.copy(deepcopy=True))

In [ ]:
δh = firedrake.interpolate(h - h_0, Q)

fig, axes = subplots()
colors = firedrake.tripcolor(δh, vmin=-100, vmax=+100, cmap="RdBu_r", axes=axes)
fig.colorbar(colors);

In [ ]:
fig, axes = subplots()
colors = firedrake.tripcolor(u, axes=axes)
fig.colorbar(colors);

On average, the ice thickened by about 5.5m over the whole domain, and about 6m at the grounding line.

In [ ]:
area = firedrake.assemble(Constant(1) * dx(mesh))
avg_δh = firedrake.assemble(δh * dx) / area
print(f"Average thickness change: {avg_δh:5.2f} m")

In [ ]:
length = firedrake.assemble(Constant(1) * ds(domain=mesh, subdomain_id=(4,)))
terminus_ids = (4,)
avg_δh_terminus = firedrake.assemble(δh * ds(terminus_ids)) / length
print(f"Average thickness change at terminus: {avg_δh_terminus:5.2f} m")

The weird transients have mostly propagated out.
There's a blob to grid east of the one nunatak, which I'm guessing is because of an incorrectly specified side wall friction.
We can fix that later if it matters.

In [ ]:
from firedrake import div

qs = [firedrake.project(div(h * u), Q) for h, u in zip(hs, us)]

In [ ]:
fig, axes = subplots()
colors = firedrake.tripcolor(qs[-1], vmin=-1.0, vmax=+1.0, cmap="RdBu", axes=axes)
fig.colorbar(colors);

The plot below shows the maximum flux divergence over time.

In [ ]:
qmaxs = np.array([abs(q.dat.data_ro).max() for q in qs])

In [ ]:
fig, axes = plt.subplots()
ts = np.linspace(0.0, final_time, num_steps + 1)
axes.set_xlabel("years")
axes.set_ylabel("meters/year")
axes.plot(ts, qmaxs);

Finally, we'll save the results to disk and continue in another notebook.

In [ ]:
with firedrake.CheckpointFile("spin_up_state.h5", "w") as chk:
    chk.save_function(C, name="friction")
    chk.save_function(h, name="thickness")
    chk.save_function(s, name="surface")
    chk.save_function(u, name="velocity")